Để xác định một sản phẩm có được xem là "được mua" hay không, ta cần dựa trên các hành động liên quan đến việc nhặt, đặt vào giỏ, trả lại và thao tác nhiều lần. Việc này đòi hỏi phải liệt kê tất cả các tình huống có thể xảy ra từ 5 cột dữ liệu, từ đó xác định khi nào sản phẩm được xem là đã mua.

Các cột liên quan bao gồm:

1. **Picking up item**: Có nhặt sản phẩm lên hay không.
2. **Returning item**: Có trả sản phẩm về kệ hay không.
3. **Putting item into bag**: Có đặt sản phẩm vào giỏ hàng hay không.
4. **Taking item out of bag**: Có lấy sản phẩm ra khỏi giỏ hàng hay không.
5. **Putting item into bag in the 2nd time**: Có đặt sản phẩm vào giỏ hàng lần thứ 2 sau khi đã trả lại hay không.

### Các tình huống có thể dẫn đến việc mua hàng:

#### Trường hợp 1:
- **Picking up item** = True: Người mua nhặt sản phẩm.
- **Returning item** = False: Không trả lại sản phẩm vào kệ.
- **Putting item into bag** = True: Sản phẩm được bỏ vào giỏ hàng.
- **Taking item out of bag** = False: Không lấy sản phẩm ra khỏi giỏ.
- **Putting item into bag in the 2nd time** = False: Không cần hành động lần thứ hai.

=> Sản phẩm được xem là mua vì nó đã được đặt vào giỏ và không bị lấy ra.

#### Trường hợp 2:
- **Picking up item** = True: Nhặt sản phẩm lên.
- **Returning item** = True: Trả sản phẩm về kệ.
- **Putting item into bag** = False: Chưa bỏ vào giỏ hàng.
- **Taking item out of bag** = False: Không lấy ra từ giỏ.
- **Putting item into bag in the 2nd time** = True: Sau đó đặt vào giỏ hàng lần thứ 2.

=> Sản phẩm được xem là mua vì nó được bỏ vào giỏ trong lần thứ 2 và không bị lấy ra.

#### Trường hợp 3:
- **Picking up item** = True: Nhặt sản phẩm lên.
- **Returning item** = False: Không trả lại sản phẩm.
- **Putting item into bag** = True: Đặt sản phẩm vào giỏ.
- **Taking item out of bag** = True: Lấy sản phẩm ra khỏi giỏ.
- **Putting item into bag in the 2nd time** = True: Sau đó đặt lại vào giỏ hàng lần thứ 2.

=> Sản phẩm được xem là mua vì nó được đặt lại vào giỏ sau khi đã bị lấy ra.

## Source code Câu 3 -   Các nhóm khách hàng theo độ tuổi mua mặt hàng nào nhiều nhất?


In [17]:
import pandas as pd

# Đọc dữ liệu từ file CSV
customer_behavior_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')
item_info_df = pd.read_csv('03_Item_Information_Data.csv')

# Chuyển đổi các cột 'Age' và 'Purchase' về kiểu số
customer_behavior_df['Age'] = pd.to_numeric(customer_behavior_df['Age'], errors='coerce')

# Xác định các sản phẩm đã được mua
customer_behavior_df['Purchased'] = (
    (customer_behavior_df['Putting item into bag'] == True) & 
    (customer_behavior_df['Taking item out of bag'] == False) & 
    (customer_behavior_df['Putting item into bag in the 2nd time'] == False) |
    (customer_behavior_df['Putting item into bag in the 2nd time'] == True)
)

# Phân loại khách hàng theo độ tuổi
def categorize_age(age):
    if 18 <= age <= 30:
        return 'Thiếu niên'
    elif 31 <= age <= 60:
        return 'Trung niên'
    elif age > 60:
        return 'Cao tuổi'
    else:
        return 'Không xác định'

customer_behavior_df['Age Group'] = customer_behavior_df['Age'].apply(categorize_age)

# Lọc các giao dịch mua hàng
purchases_df = customer_behavior_df[customer_behavior_df['Purchased'] == True]

# Tổng hợp số lượng mua hàng theo nhóm tuổi và mặt hàng
purchase_summary = purchases_df.groupby(['Age Group', 'Item ID', 'Shelf ID']).size().reset_index(name='Purchase')

# Kết hợp với thông tin sản phẩm để lấy tên sản phẩm
purchase_summary_info = pd.merge(purchase_summary, item_info_df, on=['Item ID', 'Shelf ID'], how='left')

# Xác định mặt hàng được mua nhiều nhất bởi từng nhóm tuổi
most_purchased_items = purchase_summary_info.loc[purchase_summary_info.groupby('Age Group')['Purchase'].idxmax()]

# Hiển thị kết quả
print(most_purchased_items[['Age Group', 'Name', 'Purchase']])


      Age Group                Name  Purchase
32     Cao tuổi  Bánh trứng Custard        63
166  Thiếu niên  Bánh trứng Custard        52
275  Trung niên      Kem tràng tiền       104


## Source code Câu 6 -  Tìm Top 5 các mặt hàng giảm giá được người dùng mua nhiều nhất

Để xác định nhóm khách hàng theo độ tuổi (Thiếu niên: 18 - 30; Trung niên: 31 - 60; Cao tuổi: > 60) mua mặt hàng nào nhiều nhất, chúng ta cần làm theo các bước sau:

- Đọc dữ liệu và xác định sản phẩm đã mua: Tương tự như trước, chúng ta sẽ xác định các sản phẩm đã được mua.
- Phân loại khách hàng theo độ tuổi: Thêm cột phân loại khách hàng theo độ tuổi.
- Đếm số lần mua theo nhóm tuổi và mặt hàng: Đếm số lần mua các mặt hàng theo từng nhóm tuổi.
- Xác định mặt hàng được mua nhiều nhất cho mỗi nhóm tuổi: Tìm mặt hàng được mua nhiều nhất cho từng nhóm tuổi.

In [13]:
import pandas as pd

# Đọc dữ liệu từ file CSV đã được điền đầy đủ giá trị thiếu
customer_behavior_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')
item_info_df = pd.read_csv('03_Item_Information_Data.csv')

# Xác định các sản phẩm đã được mua
customer_behavior_df['Purchased'] = (
    (customer_behavior_df['Putting item into bag'] == True) & 
    (customer_behavior_df['Taking item out of bag'] == False) & 
    (customer_behavior_df['Putting item into bag in the 2nd time'] == False) |
    (customer_behavior_df['Putting item into bag in the 2nd time'] == True)
)

# Lọc các sản phẩm đã được mua
purchased_items = customer_behavior_df[customer_behavior_df['Purchased']]

# Kết hợp dữ liệu từ hai file CSV dựa trên cả Item ID và Shelf ID
merged_df = pd.merge(purchased_items, item_info_df, on=['Item ID', 'Shelf ID'])

# Lọc các sản phẩm có giảm giá (cột Discount có khoảng trắng ở cuối)
discounted_items = merged_df[merged_df['Discount '] > 0]

# Đếm số lần mua các sản phẩm giảm giá
discounted_items_count = discounted_items.groupby(['Item ID', 'Shelf ID']).size().reset_index(name='Purchase Count')

# Kết hợp với thông tin sản phẩm để lấy tên sản phẩm
top_discounted_items = pd.merge(discounted_items_count, item_info_df, on=['Item ID', 'Shelf ID'])

# Sắp xếp và chọn ra top 5 sản phẩm giảm giá được mua nhiều nhất
top_5_discounted_items = top_discounted_items.sort_values(by='Purchase Count', ascending=False).head(5)

# Hiển thị kết quả
top_5_discounted_items[['Shelf ID', 'Item ID', 'Name', 'Purchase Count', 'Discount ']]


,Shelf ID,Item ID,Name,Purchase Count,Discount
15,0,4,Bánh trứng Custard,201,10
26,2,7,Sữa bột Milo,198,15
4,7,0,Kem tràng tiền,190,5
25,0,7,Snack khoai tây Lays,150,10
23,0,6,Bim bim Oishi,145,5


## Câu 7 -  Top 5 các mặt hàng được chạy quảng cáo được người dùng mua nhiều nhất?

Để xác định top 5 các mặt hàng được chạy quảng cáo mà người dùng mua nhiều nhất, bạn cần thực hiện các bước sau:

- Lọc các giao dịch mua hàng từ dữ liệu.
- Lọc các mặt hàng có chiến lược quảng cáo (Marketing strategy) là True.
- Đếm số lượng giao dịch mua cho mỗi mặt hàng.
- Sắp xếp và chọn ra top 5 mặt hàng có số lượng giao dịch mua nhiều nhất.

In [36]:
import pandas as pd

# Đọc dữ liệu từ file CSV với mã hóa 'utf-8'
customer_behavior_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv', encoding='utf-8')
item_info_df = pd.read_csv('03_Item_Information_Data.csv', encoding='utf-8')

# Chuyển đổi các cột 'Age' và 'Timestamp' về kiểu số và datetime
customer_behavior_df['Age'] = pd.to_numeric(customer_behavior_df['Age'], errors='coerce')
customer_behavior_df['Timestamp'] = pd.to_datetime(customer_behavior_df['Timestamp'], errors='coerce')

# Xác định các sản phẩm đã được mua
customer_behavior_df['Purchased'] = (
    (customer_behavior_df['Putting item into bag'] == True) & 
    (customer_behavior_df['Taking item out of bag'] == False) & 
    (customer_behavior_df['Putting item into bag in the 2nd time'] == False) |
    (customer_behavior_df['Putting item into bag in the 2nd time'] == True)
)

# Lọc các giao dịch mua hàng
purchases_df = customer_behavior_df[customer_behavior_df['Purchased'] == True]

# Đảm bảo các cột 'Item ID' và 'Shelf ID' có cùng kiểu dữ liệu trong cả hai DataFrame
purchases_df['Item ID'] = purchases_df['Item ID'].astype(str)
purchases_df['Shelf ID'] = purchases_df['Shelf ID'].astype(str)
item_info_df['Item ID'] = item_info_df['Item ID'].astype(str)
item_info_df['Shelf ID'] = item_info_df['Shelf ID'].astype(str)

# Kết hợp với thông tin sản phẩm để lấy thông tin chiến lược quảng cáo
purchases_with_marketing = pd.merge(purchases_df, item_info_df[['Item ID', 'Shelf ID', 'Marketing strategy']], 
                                    on=['Item ID', 'Shelf ID'], how='left')

# Lọc các mặt hàng có chiến lược quảng cáo là True
advertised_purchases = purchases_with_marketing[purchases_with_marketing['Marketing strategy'] == True]

# Tạo một cột kết hợp 'Item ID' và 'Shelf ID' để xác định duy nhất mỗi mặt hàng
advertised_purchases['Item_Shelf_ID'] = advertised_purchases['Item ID'] + '_' + advertised_purchases['Shelf ID']

# Đếm số lượng giao dịch mua cho mỗi mặt hàng
item_purchase_counts = advertised_purchases['Item_Shelf_ID'].value_counts().reset_index()
item_purchase_counts.columns = ['Item_Shelf_ID', 'Purchase Count']

# Lấy top 5 các mặt hàng được mua nhiều nhất
top_5_items = item_purchase_counts.head(5)

# Tách lại 'Item ID' và 'Shelf ID' từ 'Item_Shelf_ID'
top_5_items[['Item ID', 'Shelf ID']] = top_5_items['Item_Shelf_ID'].str.split('_', expand=True)

# Kết hợp với item_info_df để lấy tên mặt hàng và các thông tin khác
top_5_items = pd.merge(top_5_items, item_info_df[['Item ID', 'Shelf ID', 'Name', 'Marketing strategy']], 
                       on=['Item ID', 'Shelf ID'], how='left')

# Sắp xếp lại cột để dễ đọc
top_5_items = top_5_items[['Item ID', 'Shelf ID', 'Name', 'Purchase Count', 'Marketing strategy']]

# Hiển thị kết quả
print("Top 5 các mặt hàng được chạy quảng cáo được người dùng mua nhiều nhất:")
top_5_items


Top 5 các mặt hàng được chạy quảng cáo được người dùng mua nhiều nhất:


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7800\2409691610.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_df['Item ID'] = purchases_df['Item ID'].astype(str)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7800\2409691610.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_df['Shelf ID'] = purchases_df['Shelf ID'].astype(str)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7800\2409691610.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,Item ID,Shelf ID,Name,Purchase Count,Marketing strategy
0,4,0,Bánh trứng Custard,201,True
1,7,2,Sữa bột Milo,198,True
2,0,7,Kem tràng tiền,190,True
3,11,1,Khăn mặt Shine,123,True
4,15,1,Tã dán sơ sinh Goo.n Premium Newborn 70 miếng,108,True


## Câu 8 - Top 3 quầy hàng có thời lượng trung bình quan tâm đến sản phẩm, trên số lượt tương tác, là lâu nhất 

Ta tính tổng thời gian quan tâm đến sản phẩm (bao gồm cả thời gian nhìn và cầm) và sau đó tính thời gian trung bình cho mỗi quầy hàng dựa trên số lượt tương tác. Chúng ta sẽ làm điều này theo các bước sau:

- Tính tổng thời gian quan tâm đến sản phẩm cho mỗi tương tác.
- Đếm số lượt tương tác cho mỗi quầy hàng. - Số lượt tương tác ở đây sẽ được tính bằng số lượng các row  trong từng quầy hàng trong bảng Customer behavior (ví dụ sản phẩm A được nhìn và xem và ... được xem bởi 1 người ở 1 thời điểm là 1 lượt tương tác),
- Tính thời gian trung bình quan tâm đến sản phẩm cho mỗi quầy hàng bằng cách chia tổng thời gian cho số lượt tương tác.

In [37]:
import pandas as pd

# Đọc dữ liệu từ file CSV với mã hóa 'utf-8'
customer_behavior_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv', encoding='utf-8')

# Chuyển đổi các cột thời gian về kiểu số
customer_behavior_df['Looking at item (s)'] = pd.to_numeric(customer_behavior_df['Looking at item (s)'], errors='coerce')
customer_behavior_df['Holding the item (s)'] = pd.to_numeric(customer_behavior_df['Holding the item (s)'], errors='coerce')

# Tính tổng thời gian quan tâm cho mỗi tương tác
customer_behavior_df['Total Interest Time (s)'] = customer_behavior_df['Looking at item (s)'] + customer_behavior_df['Holding the item (s)']

# Đếm số lượt tương tác cho mỗi quầy hàng
interaction_counts = customer_behavior_df.groupby('Shelf ID').size().reset_index(name='Interaction Count')

# Tính tổng thời gian quan tâm cho mỗi quầy hàng
total_interest_time_by_shelf = customer_behavior_df.groupby('Shelf ID')['Total Interest Time (s)'].sum().reset_index()

# Hợp nhất hai bảng để có tổng thời gian và số lượt tương tác cho mỗi quầy hàng
shelf_interest_data = pd.merge(total_interest_time_by_shelf, interaction_counts, on='Shelf ID')

# Tính thời gian trung bình quan tâm đến sản phẩm cho mỗi quầy hàng
shelf_interest_data['Average Interest Time (s)'] = shelf_interest_data['Total Interest Time (s)'] / shelf_interest_data['Interaction Count']

# Lấy top 3 quầy hàng có thời gian trung bình quan tâm đến sản phẩm lâu nhất
top_3_shelves = shelf_interest_data.nlargest(3, 'Average Interest Time (s)')

# Hiển thị kết quả
print("Top 3 quầy hàng có thời lượng trung bình quan tâm đến sản phẩm lâu nhất:")
print(top_3_shelves[['Shelf ID', 'Average Interest Time (s)']])


Top 3 quầy hàng có thời lượng trung bình quan tâm đến sản phẩm lâu nhất:
   Shelf ID  Average Interest Time (s)
5         5                  62.260184
7         7                  61.009985
3         3                  59.845622


## Câu 9 - Top 3 quầy hàng có số sản phẩm được mua nhiều nhất?

Để xác định top 3 quầy hàng có số sản phẩm được mua nhiều nhất, chúng ta cần thực hiện các bước sau:

- Lọc các giao dịch mua thành công: Chúng ta cần xác định các giao dịch mua thành công từ dữ liệu.
- Đếm số lượng sản phẩm được mua cho mỗi quầy hàng: Sử dụng groupby để nhóm theo Shelf ID và đếm số lượng sản phẩm được mua.
- Lấy top 3 quầy hàng có số sản phẩm được mua nhiều nhất: 

In [39]:
import pandas as pd

# Đọc dữ liệu từ file CSV với mã hóa 'utf-8'
customer_behavior_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv', encoding='utf-8')

# Xác định các sản phẩm đã được mua
customer_behavior_df['Purchased'] = (
    (customer_behavior_df['Putting item into bag'] == True) & 
    (customer_behavior_df['Taking item out of bag'] == False) & 
    (customer_behavior_df['Putting item into bag in the 2nd time'] == False) |
    (customer_behavior_df['Putting item into bag in the 2nd time'] == True)
)

# Lọc các giao dịch mua thành công
successful_purchases_df = customer_behavior_df[customer_behavior_df['Purchased'] == True]

# Đếm số lượng sản phẩm được mua cho mỗi quầy hàng
purchase_counts_by_shelf = successful_purchases_df.groupby('Shelf ID').size().reset_index(name='Purchase Count')

# Lấy top 3 quầy hàng có số sản phẩm được mua nhiều nhất
top_3_shelves = purchase_counts_by_shelf.nlargest(3, 'Purchase Count')

# Hiển thị kết quả
print("Top 3 quầy hàng có số sản phẩm được mua nhiều nhất:")
print(top_3_shelves)


Top 3 quầy hàng có số sản phẩm được mua nhiều nhất:
   Shelf ID  Purchase Count
7         7            2164
0         0            1821
1         1            1740


## Câu 10 -  Người dùng có thói quen di chuyển giữa 2 quầy hàng nào nhiều nhất?

- Lọc dữ liệu theo từng Person ID: Xác định hành vi di chuyển của mỗi người dùng.

- Tìm sự thay đổi Shelf ID trong dữ liệu: Kiểm tra xem mỗi lần Shelf ID thay đổi là một lần di chuyển từ quầy này sang quầy khác.

- Tạo cặp quầy hàng: Mỗi lần có sự thay đổi từ Shelf ID A sang Shelf ID B, bạn sẽ tạo một cặp quầy hàng (A, B).

- Đếm tần suất xuất hiện của từng cặp: Đếm số lần mỗi cặp quầy hàng xuất hiện để tìm ra cặp quầy hàng mà người dùng di chuyển nhiều nhất.

In [42]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')

# Lọc ra các cột cần thiết
df = df[['Person ID', 'Shelf ID']]

# Tạo một cột mới là sự khác biệt về Shelf ID giữa các hàng liền kề của từng người
df['Previous Shelf ID'] = df.groupby('Person ID')['Shelf ID'].shift(1)

# Loại bỏ các hàng không có sự di chuyển (cùng Shelf ID)
df_movement = df.dropna(subset=['Previous Shelf ID'])
df_movement = df_movement[df_movement['Shelf ID'] != df_movement['Previous Shelf ID']]

# Tạo cặp di chuyển giữa các quầy hàng
df_movement['Movement Pair'] = list(zip(df_movement['Previous Shelf ID'], df_movement['Shelf ID']))

# Đếm số lần di chuyển giữa các cặp quầy hàng
movement_counts = df_movement['Movement Pair'].value_counts()

# In ra cặp quầy hàng có số lần di chuyển nhiều nhất
print(movement_counts)


Movement Pair
(7.0, 0)    123
(0.0, 7)    109
(1.0, 7)    104
(7.0, 1)     92
(7.0, 4)     92
(7.0, 2)     79
(2.0, 0)     77
(4.0, 7)     77
(2.0, 7)     76
(0.0, 1)     72
(1.0, 0)     71
(4.0, 1)     65
(0.0, 2)     65
(1.0, 2)     59
(1.0, 4)     57
(0.0, 3)     57
(2.0, 1)     56
(0.0, 4)     54
(4.0, 0)     54
(3.0, 7)     54
(3.0, 0)     49
(2.0, 4)     48
(0.0, 6)     46
(4.0, 2)     43
(7.0, 3)     40
(5.0, 7)     38
(7.0, 6)     37
(3.0, 1)     36
(7.0, 5)     35
(6.0, 0)     35
(4.0, 3)     34
(1.0, 6)     33
(6.0, 1)     32
(5.0, 1)     32
(1.0, 3)     32
(2.0, 3)     31
(6.0, 7)     31
(3.0, 2)     31
(6.0, 4)     28
(6.0, 2)     27
(1.0, 5)     26
(4.0, 6)     26
(5.0, 0)     26
(3.0, 4)     25
(2.0, 6)     24
(5.0, 4)     20
(0.0, 5)     19
(5.0, 2)     19
(2.0, 5)     19
(3.0, 6)     17
(4.0, 5)     16
(3.0, 5)     15
(5.0, 3)     14
(6.0, 3)     13
(6.0, 5)     11
(5.0, 6)      9
Name: count, dtype: int64


## Câu 12

In [47]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')

# Kiểm tra tên các cột
print("Các cột trong dữ liệu:", df.columns)

# Giả sử các cột cần thiết là 'Person ID', 'Age', 'Shelf ID'

# Lọc dữ liệu cho độ tuổi trung niên (31-60)
df_middle_aged = df[(df['Age'] >= 31) & (df['Age'] <= 60)]

# Đếm số lượng người dùng trung niên
num_middle_aged = df_middle_aged['Person ID'].nunique()
print(f"Số lượng người dùng độ tuổi trung niên: {num_middle_aged}")

# Tạo bảng tần suất truy cập gian hàng của người dùng trung niên
shelf_counts_middle_aged = df_middle_aged['Shelf ID'].value_counts().reset_index()
shelf_counts_middle_aged.columns = ['Shelf ID', 'Counts']

# Hiển thị top 5 gian hàng phổ biến nhất cho người dùng trung niên
print("Top 5 gian hàng phổ biến nhất cho người dùng trung niên:")
print(shelf_counts_middle_aged.head(5))


Các cột trong dữ liệu: Index(['Shelf ID', 'Person ID', 'Timestamp', 'Age', 'Gender', 'Height',
       'Weight', 'Married status', 'Moving speed (m/s)', 'Item ID',
       'Looking at item (s)', 'Holding the item (s)', 'Holding the bag',
       'Picking up item', 'Returning item', 'Putting item into bag',
       'Taking item out of bag', 'Putting item into bag in the 2nd time'],
      dtype='object')
Số lượng người dùng độ tuổi trung niên: 753
Top 5 gian hàng phổ biến nhất cho người dùng trung niên:
   Shelf ID  Counts
0         7    1626
1         0    1362
2         1    1084
3         4    1016
4         2     958


In [49]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')

# Kiểm tra tên các cột
print("Các cột trong dữ liệu:", df.columns)

# Đổi tên các cột để dễ xử lý (nếu cần)
df = df.rename(columns={
    'Shelf ID': 'Shelf_ID',
    'Looking at item (s)': 'Looking_at_item_s',
    'Holding the item (s)': 'Holding_the_item_s'
})

# Tính tổng thời gian (nhìn + giữ) cho mỗi kệ hàng
df['Total_Time'] = df['Looking_at_item_s'] + df['Holding_the_item_s']

# Tính tổng thời gian dành cho mỗi kệ hàng
total_time_per_shelf = df.groupby('Shelf_ID')['Total_Time'].sum().reset_index()
total_time_per_shelf.columns = ['Shelf_ID', 'Total_Time']

# Sắp xếp theo tổng thời gian giảm dần
total_time_per_shelf = total_time_per_shelf.sort_values(by='Total_Time', ascending=False)

# Hiển thị kết quả
print("Tổng thời gian dành cho mỗi kệ hàng:")
print(total_time_per_shelf)

# Lấy kệ hàng có tổng thời gian cao nhất
top_shelf = total_time_per_shelf.iloc[0]
top_shelf_id = top_shelf['Shelf_ID']
top_shelf_time = top_shelf['Total_Time']

print(f"\nKệ hàng có tổng thời gian nhìn và giữ cao nhất là kệ {top_shelf_id} với tổng thời gian {top_shelf_time}.")

# Đưa ra đề xuất
print("\nĐề xuất:")
print(f"Đặt kệ hàng {top_shelf_id} ở vị trí trung tâm để thu hút sự chú ý của khách hàng và tạo lợi thế cho các kệ hàng khác.")


Các cột trong dữ liệu: Index(['Shelf ID', 'Person ID', 'Timestamp', 'Age', 'Gender', 'Height',
       'Weight', 'Married status', 'Moving speed (m/s)', 'Item ID',
       'Looking at item (s)', 'Holding the item (s)', 'Holding the bag',
       'Picking up item', 'Returning item', 'Putting item into bag',
       'Taking item out of bag', 'Putting item into bag in the 2nd time'],
      dtype='object')
Tổng thời gian dành cho mỗi kệ hàng:
   Shelf_ID  Total_Time
7         7      201638
0         0      153458
1         1      120088
4         4      108934
2         2      106461
3         3       77919
5         5       47380
6         6       42914

Kệ hàng có tổng thời gian nhìn và giữ cao nhất là kệ 7 với tổng thời gian 201638.

Đề xuất:
Đặt kệ hàng 7 ở vị trí trung tâm để thu hút sự chú ý của khách hàng và tạo lợi thế cho các kệ hàng khác.


## Câu 12 - Tính toán số lượng sản phẩm được mua trong các giai đoạn có và không có chiến dịch quảng cáo hoặc khuyến mãi:



In [56]:
import pandas as pd

# Đọc dữ liệu từ file CSV
customer_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')
item_df = pd.read_csv('03_Item_Information_Data.csv')

# Đổi tên các cột để dễ xử lý
customer_df = customer_df.rename(columns={
    'Shelf ID': 'Shelf_ID',
    'Person ID': 'Person_ID',
    'Timestamp': 'Timestamp',
    'Age': 'Age',
    'Gender': 'Gender',
    'Height': 'Height',
    'Weight': 'Weight',
    'Married status': 'Married_status',
    'Moving speed (m/s)': 'Moving_speed_m_s',
    'Item ID': 'Item_ID',
    'Looking at item (s)': 'Looking_at_item_s',
    'Holding the item (s)': 'Holding_the_item_s',
    'Holding the bag': 'Holding_the_bag',
    'Picking up item': 'Picking_up_item',
    'Returning item': 'Returning_item',
    'Putting item into bag': 'Putting_item_into_bag',
    'Taking item out of bag': 'Taking_item_out_of_bag',
    'Putting item into bag in the 2nd time': 'Putting_item_into_bag_in_the_2nd_time'
})

item_df = item_df.rename(columns={
    'Shelf ID': 'Shelf_ID',
    'Item ID': 'Item_ID',
    'Name': 'Name',
    'Origin': 'Origin',
    'Location': 'Location',
    'Weight (g)': 'Weight_g',
    'Price': 'Price',
    'NSX': 'NSX',
    'HSD': 'HSD',
    'Marketing strategy': 'Marketing_strategy',
    'Discount ': 'Discount',  # Loại bỏ khoảng trắng ở cuối
    'Rate': 'Rate'
})

# Chuyển đổi cột 'Marketing_strategy' thành chuỗi và xóa khoảng trắng nếu có
item_df['Marketing_strategy'] = item_df['Marketing_strategy'].astype(str).str.strip()

# Lọc các sản phẩm có chiến dịch quảng cáo hoặc giảm giá
promoted_items_df = item_df[(item_df['Marketing_strategy'] == 'TRUE') | (item_df['Discount'] > 0)]

# Hiển thị DataFrame kết quả
print(promoted_items_df)


     Shelf_ID  Item_ID                                            Name  \
0           0        0                                       Bánh Oreo   
4           0        4                              Bánh trứng Custard   
5           0        5                        Hạt hướng dương Chacheer   
6           0        6                                   Bim bim Oishi   
7           0        7                            Snack khoai tây Lays   
9           0        9                             Socola trứng Kinder   
10          0       10                               Kẹo dẻo Alphelibe   
13          1        0                                    Dầu gội XMen   
14          1        1                                  Dầu gội Romano   
15          1        2                                    Dầu gội Dove   
16          1        3                                 Dầu gội Sunsilk   
17          1        4                                 Sửa rửa mặt Oxy   
21          1        8                

In [55]:
import pandas as pd

# Đọc dữ liệu từ file CSV
customer_df = pd.read_csv('03_Customer_Behavior_Data_Filled.csv')
item_df = pd.read_csv('03_Item_Information_Data.csv')

# Đổi tên các cột để dễ xử lý
customer_df = customer_df.rename(columns={
    'Shelf ID': 'Shelf_ID',
    'Person ID': 'Person_ID',
    'Timestamp': 'Timestamp',
    'Age': 'Age',
    'Gender': 'Gender',
    'Height': 'Height',
    'Weight': 'Weight',
    'Married status': 'Married_status',
    'Moving speed (m/s)': 'Moving_speed_m_s',
    'Item ID': 'Item_ID',
    'Looking at item (s)': 'Looking_at_item_s',
    'Holding the item (s)': 'Holding_the_item_s',
    'Holding the bag': 'Holding_the_bag',
    'Picking up item': 'Picking_up_item',
    'Returning item': 'Returning_item',
    'Putting item into bag': 'Putting_item_into_bag',
    'Taking item out of bag': 'Taking_item_out_of_bag',
    'Putting item into bag in the 2nd time': 'Putting_item_into_bag_in_the_2nd_time'
})

item_df = item_df.rename(columns={
    'Shelf ID': 'Shelf_ID',
    'Item ID': 'Item_ID',
    'Name': 'Name',
    'Origin': 'Origin',
    'Location': 'Location',
    'Weight (g)': 'Weight_g',
    'Price': 'Price',
    'NSX': 'NSX',
    'HSD': 'HSD',
    'Marketing strategy': 'Marketing_strategy',
    'Discount ': 'Discount',  # Loại bỏ khoảng trắng ở cuối
    'Rate': 'Rate'
})

# Chuyển đổi cột 'Timestamp' thành định dạng datetime
customer_df['Timestamp'] = pd.to_datetime(customer_df['Timestamp'])

# Chuyển đổi cột 'Marketing_strategy' thành chuỗi và xóa khoảng trắng nếu có
item_df['Marketing_strategy'] = item_df['Marketing_strategy'].astype(str).str.strip()

# Lọc các sản phẩm có chiến dịch quảng cáo hoặc giảm giá
promoted_items_df = item_df[(item_df['Marketing_strategy'] == 'TRUE') | (item_df['Discount'] > 0)]

# Xác định các sản phẩm đã được mua
customer_df['Purchased'] = (
    (customer_df['Putting_item_into_bag'] == True) & 
    (customer_df['Taking_item_out_of_bag'] == False) & 
    (customer_df['Putting_item_into_bag_in_the_2nd_time'] == False) |
    (customer_df['Putting_item_into_bag_in_the_2nd_time'] == True)
)

# Kết hợp dữ liệu từ hai file dựa trên 'Item_ID'
merged_df = pd.merge(customer_df, item_df, on='Item_ID', how='left', suffixes=('', '_item'))

# Thêm cột 'Promotion' vào dữ liệu khách hàng, kiểm tra Discount hoặc Marketing_strategy
merged_df['Promotion'] = (merged_df['Marketing_strategy'] == 'TRUE') | (merged_df['Discount'] > 0)

# Tính toán số lượng sản phẩm được mua trong các giai đoạn có và không có chiến dịch quảng cáo hoặc khuyến mãi
purchased_with_promotion = merged_df[(merged_df['Promotion'] == True) & (merged_df['Purchased'] == True)].shape[0]
purchased_without_promotion = merged_df[(merged_df['Promotion'] == False) & (merged_df['Purchased'] == True)].shape[0]

# Hiển thị kết quả
print("Số lượng sản phẩm được mua trong các giai đoạn có chiến dịch quảng cáo hoặc khuyến mãi:", purchased_with_promotion)
print("Số lượng sản phẩm được mua trong các giai đoạn không có chiến dịch quảng cáo hoặc khuyến mãi:", purchased_without_promotion)

# Đưa ra nhận định và đề xuất
if purchased_with_promotion > purchased_without_promotion:
    print("\nChiến dịch quảng cáo hoặc khuyến mãi có hiệu quả trong việc tăng số lượng sản phẩm được mua.")
else:
    print("\nChiến dịch quảng cáo hoặc khuyến mãi không có hiệu quả rõ rệt trong việc tăng số lượng sản phẩm được mua.")


Số lượng sản phẩm được mua trong các giai đoạn có chiến dịch quảng cáo hoặc khuyến mãi: 32129
Số lượng sản phẩm được mua trong các giai đoạn không có chiến dịch quảng cáo hoặc khuyến mãi: 42498

Chiến dịch quảng cáo hoặc khuyến mãi không có hiệu quả rõ rệt trong việc tăng số lượng sản phẩm được mua.
